In [192]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [193]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'

In [194]:
df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)
df_houses_pool = pd.read_csv(pathdata + 'data_output/Base_pool_hogares_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)

In [195]:
## Defino funciones dimension mercado de trabajo e ingresos

# Componente: Inserción laboral

# T1. 
# Variable: Actividad productiva
# Indicador: Ratio M/V en tasa de empleo

def actividad_productiva(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):
    
    """
    Calcula la actividad productiva, a partir de la proporción de tasas de empleo entre hombres y mujeres, junto con los errores asociados.

    Args:
        df (DataFrame): DataFrame de entrada que contiene los datos de la EPH.
        tipo (str): Tipo de encuesta de la EPH, 'Aglomerado' o 'Urbano'. Por defecto: 'Aglomerado'.
        base (str): Tipo de base de la EPH, 'Individual' o 'Hogar'. Por defecto: 'Individual'.
        confidence_level (float): Nivel de confianza para el cálculo de errores. Por defecto: 0.95.

    Returns:
        tasa (DataFrame): DataFrame con las tasas de empleo (M/V) desglosadas por Aglomerado o Provincia. Expresado en % [0-100]
        size (DataFrame): DataFrame con los numeros de observaciones utilizados para calcular tasas de empleo (M/V). Expresado en valores absolutos
        ratio (DataFrame): DataFrame con las proporciones de tasas de empleo (M/V). Expresado en % [0-100]
        error (DataFrame): DataFrame con los errores asociados a las proporciones. Expresado en % [0-100]. 
                            LI: Limite inferior, LS: Limite superior, ME: Margen de error, ER: Error relativo (CV)


    """
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe contener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    numerador = df_temp[df_temp['ESTADO'] == 1].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)
    denominador = df_temp.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_temp[df_temp['ESTADO'] == 1].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Actividad productiva'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [196]:
tasa, size, ratio, error = actividad_productiva(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)

empleo = pd.concat([tasa, size, ratio, error], axis=1)

In [200]:
empleo

,Varon,Mujer,N_v,N_m,Actividad productiva,LI,LS,ME,ER
PROVINCIA,,,,,,,,,
2,78.710304,70.957452,642,594,90.150144,85.317535,94.982752,4.832609,5.360622
6,75.153672,55.778992,3985,3250,74.219915,72.048389,76.391442,2.171526,2.925800
10,69.529090,52.433530,1026,813,75.412363,70.971985,79.852740,4.440377,5.888129
14,79.622598,54.541412,1906,1487,68.499915,65.389416,71.610414,3.110499,4.540880
18,72.778626,48.508609,924,661,66.652274,61.882320,71.422227,4.769953,7.156475
22,73.566052,44.943799,829,608,61.093124,56.128739,66.057509,4.964385,8.125930
26,73.485409,55.669633,1176,973,75.756036,71.742573,79.769499,4.013463,5.297879
30,73.334174,51.223948,1496,1100,69.850038,66.142374,73.557702,3.707664,5.308034
34,64.272893,32.303294,841,459,50.259593,44.893889,55.625296,5.365703,10.675979


In [197]:
# Componente: Inserción laboral

# T2. 
# Variable: Busqueda de trabajo
# Indicador: Ratio M/V en tasa de desempleo

def busqueda_trabajo(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):
    
    """
    Calcula la actividad productiva, a partir de la proporción de tasas de desempleo entre hombres y mujeres, junto con los errores asociados.

    Args:
        df (DataFrame): DataFrame de entrada que contiene los datos de la EPH.
        tipo (str): Tipo de encuesta de la EPH, 'Aglomerado' o 'Urbano'. Por defecto: 'Aglomerado'.
        base (str): Tipo de base de la EPH, 'Individual' o 'Hogar'. Por defecto: 'Individual'.
        confidence_level (float): Nivel de confianza para el cálculo de errores. Por defecto: 0.95.

    Returns:
        tasa (DataFrame): DataFrame con las tasas de desempleo (M/V) desglosadas por Aglomerado o Provincia. Expresado en % [0-100]
        size (DataFrame): DataFrame con los numeros de observaciones utilizados para calcular tasas de desempleo (M/V). Expresado en valores absolutos
        ratio (DataFrame): DataFrame con las proporciones de tasas de desempleo (M/V). Expresado en % [0-100]
        error (DataFrame): DataFrame con los errores asociados a las proporciones. Expresado en % [0-100]. 
                            LI: Limite inferior, LS: Limite superior, ME: Margen de error, ER: Error relativo (CV)


    """
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe contener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    numerador = df_temp[df_temp['ESTADO'] == 2].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)
    df_estado = df_temp[(df_temp['ESTADO'] == 1) | (df_temp['ESTADO'] == 2)]
    denominador = df_estado.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_temp[df_temp['ESTADO'] == 2].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Varon']
    p2 = tasa['Mujer']
    n1 = size['N_v']
    n2 = size['N_m']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Búsqueda de trabajo'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [198]:
tasa, size, ratio, error = busqueda_trabajo(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)

desempleo = pd.concat([tasa, size, ratio, error], axis=1)

In [199]:
desempleo

,Varon,Mujer,N_v,N_m,Búsqueda de trabajo,LI,LS,ME,ER
PROVINCIA,,,,,,,,,
2,6.270256,5.029093,39,36,124.679654,114.246343,135.112966,10.433311,8.368095
6,8.252182,10.440042,301,342,79.043576,74.553036,83.534116,4.490540,5.681094
10,4.901536,8.311059,55,59,58.976074,49.911190,68.040958,9.064884,15.370443
14,6.333984,12.016204,129,164,52.712020,46.198097,59.225943,6.513923,12.357567
18,4.921424,4.541953,43,27,108.354805,98.181874,118.527737,10.172931,9.388537
22,4.864433,4.154527,44,27,117.087538,107.235796,126.939280,9.851742,8.413997
26,4.906585,3.513886,71,43,139.634140,132.182078,147.086201,7.452062,5.336848
30,5.460783,10.521391,94,96,51.901716,44.235584,59.567849,7.666133,14.770480
34,5.816408,2.355315,50,13,246.948214,236.457868,257.438559,10.490346,4.247994
